#Model to Classify whether Total discount is zero or not

Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import plot_importance
from matplotlib import pyplot

Upload "Data_Encoded.csv" from the INPUT folder provided in the drive link

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
data_c = pd.read_csv('/content/Data_Encoded.csv')
data_c.head()

,total_volume_2019,total_volume_2018,city_tier,gross_turnover_2019,on_invoice_discount,off_invoice_discount,product_volume_2019,tax,total_discount,gto-tax,Premium,Entertainment Led,Food Led,Institutional,Not applicable,Wholesaler,BULK,CAN,KEG,PERFECTDRAFT,RETURNABLE,sub_segment_Bar,sub_segment_Hybrid,sub_segment_Beer bar,sub_segment_Restaurant,sub_segment_Institutional,sub_segment_Sports Venue,brand_LEFFE,brand_JUPILER,brand_HOEGAARDEN,brand_TRIPEL KARMELIET,brand_BELLE VUE,brand_STELLA ARTOIS,sub_brand_JUPILER PILS,sub_brand_LEFFE BLONDE,sub_brand_HOEGAARDEN WHITE,"sub_brand_JUPILER 0,0",sub_brand_LEFFE BRUNE,sub_brand_TRIPEL KARMELIET,sub_brand_STELLA ARTOIS REGULAR,sub_brand_HOEGAARDEN ROSEE,province_West Flanders,province_Brussels Capital Region,province_Liège,province_Flemish Brabant,province_East Flanders,province_Hainaut,province_Antwerp,province_Limburg,province_Namur
0,0.48,0.557,1,141.20,0.00,0.0,0.48,12.23,0.00,128.97,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0.45,0.540,2,113.65,0.00,0.0,0.45,7.85,0.00,105.80,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,270.97,225.720,2,61551.06,14511.46,0.0,238.57,3577.69,14511.46,57973.37,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,270.97,225.720,2,6001.00,1425.19,180.0,20.40,519.41,1605.19,5481.59,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,270.97,225.720,2,3051.36,718.32,0.0,12.00,288.63,718.32,2762.73,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Creating a column which tells if the poc got a total discount or not

In [ ]:
data_c['Tdiscount']=0

In [ ]:
for i in range(len(data_c)):
  if data_c['total_discount'][i]>0:
    data_c['Tdiscount'][i]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_c.head(10)

,total_volume_2019,total_volume_2018,city_tier,gross_turnover_2019,on_invoice_discount,off_invoice_discount,product_volume_2019,tax,total_discount,gto-tax,Premium,Entertainment Led,Food Led,Institutional,Not applicable,Wholesaler,BULK,CAN,KEG,PERFECTDRAFT,RETURNABLE,sub_segment_Bar,sub_segment_Hybrid,sub_segment_Beer bar,sub_segment_Restaurant,sub_segment_Institutional,sub_segment_Sports Venue,brand_LEFFE,brand_JUPILER,brand_HOEGAARDEN,brand_TRIPEL KARMELIET,brand_BELLE VUE,brand_STELLA ARTOIS,sub_brand_JUPILER PILS,sub_brand_LEFFE BLONDE,sub_brand_HOEGAARDEN WHITE,"sub_brand_JUPILER 0,0",sub_brand_LEFFE BRUNE,sub_brand_TRIPEL KARMELIET,sub_brand_STELLA ARTOIS REGULAR,sub_brand_HOEGAARDEN ROSEE,province_West Flanders,province_Brussels Capital Region,province_Liège,province_Flemish Brabant,province_East Flanders,province_Hainaut,province_Antwerp,province_Limburg,province_Namur,Tdiscount
0,0.48,0.557,1,141.20,0.00,0.00,0.48,12.23,0.00,128.97,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0.45,0.540,2,113.65,0.00,0.00,0.45,7.85,0.00,105.80,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,270.97,225.720,2,61551.06,14511.46,0.00,238.57,3577.69,14511.46,57973.37,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,270.97,225.720,2,6001.00,1425.19,180.00,20.40,519.41,1605.19,5481.59,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,270.97,225.720,2,3051.36,718.32,0.00,12.00,288.63,718.32,2762.73,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
5,29.61,25.860,4,104.10,0.00,0.00,0.30,1.55,0.00,102.55,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,29.61,25.860,4,176.50,0.00,0.00,0.60,15.27,0.00,161.23,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,29.61,25.860,4,2227.65,0.00,686.13,11.34,61.37,686.13,2166.28,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
8,29.61,25.860,4,3412.19,0.00,1050.98,17.37,94.12,1050.98,3318.07,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
9,0.42,0.505,4,20.82,6.15,0.00,0.06,0.31,6.15,20.51,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


Dropping redundant columns/ Getting the data ready to train

In [ ]:
data_c.drop(['total_discount', 'on_invoice_discount', 'off_invoice_discount', 'gross_turnover_2019','tax'], axis = 1, inplace = True)

In [ ]:
data_c['2019-2018'] = data_c['total_volume_2019']-data_c['total_volume_2018']

In [ ]:
data_c.drop('total_volume_2018', axis = 1, inplace = True)

Splitting the data for training and testing

In [ ]:
X_tdc = data_c.drop('Tdiscount', axis = 1)
Y_tdc = data_c['Tdiscount']

In [ ]:
X_tdc_train, X_tdc_test, y_tdc_train, y_tdc_test = train_test_split(X_tdc,Y_tdc,test_size = 0.2, random_state = 10, stratify = Y_tdc)

Model to check if he gets Tdiscount = 0 or 1

Final training model after hyperparameter tuning


In [ ]:
XGBc = XGBClassifier(base_score=0.5, booster='gbtree', boosting='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0.15, learning_rate=0.15, max_delta_step=0, max_depth=30,
              min_child_weight=2, missing=None, n_estimators=575, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.8, scale_pos_weight=5.0, seed=None,
              silent=None, subsample=0.95, verbosity=1,scoring ='roc_auc')
XGBc.fit(X_tdc_train,y_tdc_train, eval_set= [(X_tdc_test,y_tdc_test)], early_stopping_rounds=10, eval_metric= 'aucpr')

[0]	validation_0-aucpr:0.93291
Will train until validation_0-aucpr hasn't improved in 10 rounds.
[1]	validation_0-aucpr:0.935169
[2]	validation_0-aucpr:0.943657
[3]	validation_0-aucpr:0.951818
[4]	validation_0-aucpr:0.955517
[5]	validation_0-aucpr:0.960568
[6]	validation_0-aucpr:0.960162
[7]	validation_0-aucpr:0.968762
[8]	validation_0-aucpr:0.970574
[9]	validation_0-aucpr:0.971518
[10]	validation_0-aucpr:0.972071
[11]	validation_0-aucpr:0.974275
[12]	validation_0-aucpr:0.974594
[13]	validation_0-aucpr:0.975351
[14]	validation_0-aucpr:0.9766
[15]	validation_0-aucpr:0.977776
[16]	validation_0-aucpr:0.977455
[17]	validation_0-aucpr:0.977633
[18]	validation_0-aucpr:0.977725
[19]	validation_0-aucpr:0.979018
[20]	validation_0-aucpr:0.978533
[21]	validation_0-aucpr:0.978748
[22]	validation_0-aucpr:0.978797
[23]	validation_0-aucpr:0.978982
[24]	validation_0-aucpr:0.979296
[25]	validation_0-aucpr:0.979902
[26]	validation_0-aucpr:0.980604
[27]	validation_0-aucpr:0.981087
[28]	validation_0-aucpr

XGBClassifier(base_score=0.5, booster='gbtree', boosting='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0.15, learning_rate=0.15, max_delta_step=0, max_depth=30,
              min_child_weight=2, missing=None, n_estimators=575, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.8, scale_pos_weight=5.0,
              scoring='roc_auc', seed=None, silent=None, subsample=0.95,
              verbosity=1)

Predicting test dataset

In [ ]:
XGBcpred = XGBc.predict(X_tdc_test)

Accuracy score

In [ ]:
metrics.accuracy_score(y_tdc_test, XGBcpred)

0.9167847481643694

Confusion matrix

In [ ]:
metrics.confusion_matrix(y_tdc_test, XGBcpred)

Precision and recall scores

In [ ]:
print(metrics.precision_score(y_tdc_test, XGBcpred))
print(metrics.recall_score(y_tdc_test, XGBcpred))

0.9173306772908366
0.9900184275184275


Testing training dataset ( commented out )

In [ ]:
#XGBcpred = XGBc.predict(X_tdc_train)

In [ ]:
#metrics.accuracy_score(y_tdc_train, XGBcpred)

0.9989693709942349

In [ ]:
#metrics.confusion_matrix(y_tdc_train, XGBcpred)

array([[ 4973,    32],
       [    0, 26044]])

Downloading the trained model as pickle file ( commented )

In [ ]:
#import pickle
#with open('XGB classify total discount.pickle','wb') as modelFile:
#     pickle.dump(XGBc,modelFile)

Feature importance of the model

Patrial tree representation

#Hyper parameter tuning of XGB ( commented out )

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# n_estimators = [int(x) for x in np.linspace(start = 500, stop=600, num = 5)]
# learning_rate = [0.15,0.16,0.17,0.18]
# boosting = ['gbtree']
# max_depth = [int(x) for x in np.linspace(20,50,10)]
# min_child_weight = [2]
# scale_pos_weight = [5]
# gamma = [0.15,0.35,0.5]
# reg_lambda = [0.8,1.0,1.5]
# random_grid = {'n_estimators': n_estimators,
#                'learning_rate': learning_rate,
#                'boosting': boosting,
#                'max_depth': max_depth,
#                'min_child_weight': min_child_weight,
#                'scale_pos_weight': scale_pos_weight,
#                'gamma': gamma,
#                'reg_lambda': reg_lambda}
# print(random_grid)

{'n_estimators': [500, 525, 550, 575, 600], 'learning_rate': [0.15, 0.16, 0.17, 0.18], 'boosting': ['gbtree'], 'max_depth': [20, 23, 26, 30], 'min_child_weight': [2], 'scale_pos_weight': [5], 'gamma': [0.15, 0.35, 0.5], 'reg_lambda': [0.8, 1.0, 1.5]}


In [ ]:
# h_xgb = XGBClassifier()
# h_xgb_rscv = RandomizedSearchCV(estimator=h_xgb, param_distributions= random_grid,n_iter=5, cv=3,verbose =1, random_state=69)
# h_xgb_rscv.fit(X_tdc_train,y_tdc_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 19.0min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, sc...
                   iid='deprecated', n_iter=5, n_jobs=None,
                   param_distributions={'boosting': ['gbtree'],
                                        'gamma': [0.1

Used this to train the final model

In [ ]:
# h_xgb_rscv.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', boosting='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0.15, learning_rate=0.15, max_delta_step=0, max_depth=30,
              min_child_weight=2, missing=None, n_estimators=575, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.8, scale_pos_weight=5, seed=None,
              silent=None, subsample=1, verbosity=1)

#Trying other classifiers ( Commented out )

In [ ]:
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB

In [ ]:
# log_clf = LogisticRegression()
# rf_clf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                        criterion='gini', max_depth=317, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=2, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=251,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)
# svm_clf = SVC()
# nb_clf = GaussianNB()

In [ ]:
# for clf in (log_clf, rf_clf, svm_clf,nb_clf):
#   clf.fit(X_tdc_train,y_tdc_train)
#   ypred = clf.predict(X_tdc_test)
#   print(clf.__class__.__name__, metrics.accuracy_score(y_tdc_test,ypred))

LogisticRegression 0.8388509596805359
RandomForestClassifier 0.8695092103568208
SVC 0.8388509596805359
GaussianNB 0.4906608269998712


Voting classifier

In [ ]:
# evc = VotingClassifier(estimators= [('rf',rf_clf),('xgb',XGBc)], voting = 'hard')

In [ ]:
# evc.fit(X_tdc_train,y_tdc_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=317,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=2,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
         

In [ ]:
# metrics.accuracy_score(y_tdc_test, evc.predict(X_tdc_test))

0.8621666881360299

In [ ]:
# metrics.confusion_matrix(y_tdc_test, evc.predict(X_tdc_test))

array([[ 247, 1004],
       [  66, 6446]])